# Capstone Project: Segmenting and Clustering Neighborhoods in Toronto

## Part 2 - Data Enrichment - adding Latitude and Longitude

#### import required modules:

In [1]:
import pandas as pd
import json
import requests # library to handle requests
import time # the data source has a max calls per second so the loop needs to be slowed
import geocoder


---
In the next section I used a free account at LocationIQ and their API to find the coordinates using the python geocoder. The API was stable mostly but I've used a "Try/Catch" setup, a time delay as per the free account documenation on request frequency and also manually adjust several of the neighbourhoods.

**NOTE**
due to the fact that I have verified the data using maps my row counts my be different to any provided in the course, however this doesnt imapct the analysis and I have verified my results are current.

In [2]:
# read in the csv with 1 neighbourhood per row
df_tpc=pd.read_csv('TNC_UnpackN.csv').drop('Unnamed: 0', axis=1)

# the next few lines remove some rows that caused problems with getting the coords:
df_tpc.drop(df_tpc[df_tpc['Neighbourhood'].str.contains("Processing")].index, inplace=True) #dropping mail plants

# there is a white space in this field... its annoying so I'm removing it
df_tpc['Neighbourhood'] = df_tpc['Neighbourhood'].apply(lambda n: n.strip())

# verfy the data types
df_tpc.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

In [3]:
# these lines are just for rerunning purposes:
# they removes old versions (if they exist) of the lat/lon and reset to 0

try:
    df_tpc.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
except:
    pass
    
df_tpc['Latitude']=[0.000000]*df_tpc.shape[0]
df_tpc['Longitude']=[0.000000]*df_tpc.shape[0]
df_tpc.reset_index(drop=True, inplace=True)
df_tpc

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,Regent Park,0.0,0.0
3,M5A,Downtown Toronto,Harbourfront,0.0,0.0
4,M6A,North York,Lawrence Manor,0.0,0.0
...,...,...,...,...,...
209,M8Z,Etobicoke,Mimico NW,0.0,0.0
210,M8Z,Etobicoke,The Queensway West,0.0,0.0
211,M8Z,Etobicoke,South of Bloor,0.0,0.0
212,M8Z,Etobicoke,Kingsway Park South West,0.0,0.0


In [4]:
for n in range(0, df_tpc.shape[0]):
    if df_tpc.iat[n, 3]<43 or df_tpc.iat[n, 3]>44:
        try:
            address=df_tpc.iat[n, 2]+', '+df_tpc.iat[n, 1]+', Toronto, Ontario, Canada'
            g = geocoder.locationiq(address, key='0dd3eb043b17af')

            if g.lat<43 or g.lat>44:
                address=df_tpc.iat[n, 2]+', '+df_tpc.iat[n, 1]+', Ontario, Canada'+', '+df_tpc.iat[n, 0]
                g = geocoder.locationiq(address, key='0dd3eb043b17af')            

            if g.lat<43 or g.lat>44:
                address=df_tpc.iat[n, 2]+', '+df_tpc.iat[n, 1]+', Ontario, Canada'+', '+df_tpc.iat[n, 0]
                g = geocoder.locationiq(address, key='0dd3eb043b17af')            

            df_tpc.iat[n, 3]=g.lat
            df_tpc.iat[n, 4]=g.lng
        except:
            df_tpc.iat[n, 3]=100
            df_tpc.iat[n, 4]=100
            print(n, 'issue')    
        time.sleep(0.6)
        print(n,df_tpc.iat[n, 2], g.latlng, )


0 Parkwoods [43.7587999, -79.3201966]
1 Victoria Village [43.732658, -79.3111892]
2 Regent Park [43.659738, -79.361559]
3 Harbourfront [43.654652, -79.381164]
4 Lawrence Manor [43.7220788, -79.4375067]
5 Lawrence Heights [43.7227784, -79.4509332]
6 Queen's Park [43.659897, -79.390337]
7 Ontario Provincial Government [43.631212, -79.426197]
8 Islington Avenue [43.6782071, -79.5382694]
9 Humber Valley Village [43.6664717, -79.5243136]
10 Malvern [43.8091955, -79.2217008]
11 Rouge [43.8049304, -79.1658374]
12 Don Mills [43.775347, -79.3459439]
13 Parkview Hill [43.704479, -79.32949]
14 Woodbine Gardens [43.7120785, -79.3025673]
15 Garden District [43.6523481, -79.3789918]
16 Ryerson [43.65846945, -79.3789932724589]
17 Glencairn [43.7087117, -79.4406853]
18 West Deane Park [43.6631995, -79.5685684]
19 Princess Gardens [43.67061, -79.553568]
20 Martin Grove [43.694774, -79.572406]
21 Islington [43.645335, -79.5248163]
22 Cloverdale [43.633637, -79.5497447]
23 Rouge Hill [43.7802711, -79.130

191 West Humber - Clairville [43.72337025, -79.5974574109517]
192 Upper Rouge [43.8049304, -79.1658374]
193 St. James Town [34.012293, -117.644491]
194 Cabbagetown [43.654652, -79.381164]
195 First Canadian Place [43.626433, -79.627614]
196 Underground city [45.499533, -73.565866]
197 The Kingsway [43.6473811, -79.5113328]
198 Montgomery Road [43.6490313, -79.5188556]
199 Old Mill North [43.6507115, -79.4928714]
200 Church and Wellesley [43.6614803, -79.3829401]
201 Old Mill South [43.649826, -79.4943338]
202 King's Mill Park [43.6510895, -79.4920291]
203 Sunnylea [43.64159705, -79.5016654011084]
204 Humber Bay [43.6400463, -79.495028]
205 Mimico NE [43.6166773, -79.4968048]
206 The Queensway East [43.6093093, -79.5677317]
207 Royal York South East [43.6481827, -79.5112961]
208 Kingsway Park South East [43.6503362, -79.49999]
209 Mimico NW [43.6166773, -79.4968048]
210 The Queensway West [43.6340452, -79.4748634]
211 South of Bloor [43.6480098, -79.5091563]
212 Kingsway Park South West

---
The next section manually corrects the 3 errors

In [5]:
# name correction required for neighbourhood - "ST W" added to the name
n=51
address=df_tpc.iat[n, 2]+' ST W , Toronto, ON, Canada, '+df_tpc.iat[n, 0]
g = geocoder.locationiq(address, key='0dd3eb043b17af')
df_tpc.iat[n, 3]=g.lat
df_tpc.iat[n, 4]=g.lng
print(address, g.latlng, g.postal)

Adelaide ST W , Toronto, ON, Canada, M5H [43.6493496, -79.3833957] M5H 1T1


In [6]:
# name correction required for neighbourhood - India Bazaar is a restaurant / Eating area in Little India
n=86
address='Little India, ON, Canada'
g = geocoder.locationiq(address, key='0dd3eb043b17af')
df_tpc.iat[n, 3]=g.lat
df_tpc.iat[n, 4]=g.lng
print(address, g.latlng, g.postal)

Little India, ON, Canada [43.6722231, -79.3235032] M4L 1Z4


In [7]:
# name correction required for neighbourhood - Road name needed to specifiy post code
n=135
address='Spadina Road, Toronto , ON, '+df_tpc.iat[n, 0]+' Canada'
g = geocoder.locationiq(address, key='0dd3eb043b17af')
df_tpc.iat[n, 3]=g.lat
df_tpc.iat[n, 4]=g.lng
print(address, g.latlng, g.postal)

Spadina Road, Toronto , ON, M5R Canada [43.673333, -79.406464] M5R 2T9


---

In [8]:
df_tpc.dropna(inplace=True)
df_tpc.reset_index(drop=True, inplace=True)
df_tpc

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.659738,-79.361559
3,M5A,Downtown Toronto,Harbourfront,43.654652,-79.381164
4,M6A,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...,...
209,M8Z,Etobicoke,Mimico NW,43.616677,-79.496805
210,M8Z,Etobicoke,The Queensway West,43.634045,-79.474863
211,M8Z,Etobicoke,South of Bloor,43.648010,-79.509156
212,M8Z,Etobicoke,Kingsway Park South West,43.650336,-79.499990


---
Saving the data as a CSV to use later and avoid haivng to re-create the dataset repeatedly

In [9]:
df_tpc.to_csv('TNC_myLatLon.csv', sep = ',', header=df_tpc.columns)